In [45]:
#Importamos todas las librerias necesarias que serviran desde poder cargar el CSV, hasta poder procesar el texto, aplicar el entrenamiento y graficar
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#definir las stopwords cargadas por la libreria
nltk.download('stopwords')

In [17]:
#cargar el csv y definir las stopwords
df = pd.read_csv('straight_outta_compton_reviews.csv')

#convertir todas las reseñas a cadenas
df['review'] = df['review'].astype(str)

stop_words = set(stopwords.words('english'))

In [20]:
#Creamos al funcion de procesamiento de texto donde quitaremoa caracteres innecesarios, convertiremos a minusculas, dividiremos el texto y eliminaremos stopwords
def preprocess_text(text):
    if isinstance(text, str):  # Asegurarse de que el texto es una cadena
        # Eliminar caracteres especiales
        text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ\s]', '', text)
        # Convertir a minúsculas
        text = text.lower()
        # Tokenizar y eliminar stop words
        text = ' '.join(word for word in text.split() if word not in stop_words)
        return text
    return ''

In [21]:
#aplicar el procesamiento para las reseñas
df['cleaned_review'] = df['review'].apply(preprocess_text)

In [ ]:
print(df.head)

In [ ]:
#creamos el nuevo csv con el texto procesado
df.to_csv('straight_outta_compton_reviews_with_labels.csv', index=False)
print("Reseñas procesadas guardadas en 'straight_outta_compton_reviews_with_labels.csv'.")

In [ ]:
#cargamos el nuevo csv
df = pd.read_csv('straight_outta_compton_reviews_with_labels.csv')

#vemos los primeros registros para corroborar
print(df.head())

In [29]:
#crearemos la funcion para obtener etiquetas de sentimiento
def get_sentiment(label):

    #con libreria textblob analizamos el sentimiento
    analysis = TextBlob(label)

    if analysis.sentiment.polarity > 0:
        return 1 #positiva
    else:
        return 0 #negativa
    
df['label'] = df['cleaned_review'].apply(get_sentiment)

In [32]:
#dividiremos los datos en dos conjuntos: entrenamiento y prueba

X = df['cleaned_review'] #reseñas procesadas
Y = df['label'] #etiquetas de positivo y negativo

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state=42)



In [ ]:
#convertiremos al texto a vectores numericos para poder entrenar el modelo
vectorizer = CountVectorizer()

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

#Entrenar el modelo de regresion logistica
model = LogisticRegression()
model.fit(X_train_vec, y_train)

In [ ]:
#Ahora evaluaremos el modelo con el conjunto de prueba para su precision
y_pred = model.predict(X_test_vec)

#Evaluaremos la precision del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precision del modelo: {accuracy * 100:.2f}%")

#Mostrar el reporte de clasificacion
print(classification_report(y_test, y_pred))

In [ ]:
#Creamos matriz de confusion y la mostramos
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negativo', 'Positivo'], yticklabels=['Negativo', 'Positivo'])
plt.xlabel('Prediccion')
plt.ylabel('Real')
plt.title('Matriz de confusion')
plt.show()

In [ ]:
# Crear un nuevo DataFrame con las reseñas, etiquetas reales y predicciones
df_results = pd.DataFrame({
    'review': X_test,
    'label': y_test,
    'predicted_label': y_pred
})

# Guardar los resultados en un nuevo CSV
df_results.to_csv('straight_outta_compton_classification_results.csv', index=False)
print("Resultados guardados en 'straight_outta_compton_classification_results.csv'.")

## Los datos guardados en el nuevo CSV se importaran a power BI donde se graficaran los resultados de la prediccion.